<a href="https://colab.research.google.com/github/zwt4pb/dsprojectjterm2/blob/main/streams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

In [5]:
url= "https://kworb.net/spotify/artist/1Xyo4u8uXC1ZmMpatF05PJ_songs.html"

response = requests.get(url)
if response.status_code == 200:
    print("Page retrieved successfully!")
else:
    print("Failed to retrieve the page.")
    exit()


Page retrieved successfully!


scraping songs and streams now

In [11]:
titles_selector = ".text a"
streams_selector = ".sortable td:nth-child(2)"

titles = soup.select(titles_selector)
streams = soup.select(streams_selector)

streaming_data = []
for title, stream in zip(titles, streams):
    streaming_data.append({
        "song_name": title.get_text(strip=True),
        "streams": stream.get_text(strip=True)
    })

for item in streaming_data[:10]:
    print(f"Song: {item['song_name']}, Streams: {item['streams']}")


Song: Blinding Lights, Streams: 4,646,503,363
Song: Starboy, Streams: 3,673,471,314
Song: Die For You, Streams: 2,594,175,849
Song: The Hills, Streams: 2,522,521,678
Song: Save Your Tears, Streams: 2,219,167,728
Song: Call Out My Name, Streams: 1,979,781,877
Song: I Feel It Coming, Streams: 1,873,612,979
Song: Can't Feel My Face, Streams: 1,873,217,850
Song: Earned It (Fifty Shades Of Grey), Streams: 1,805,687,145
Song: Save Your Tears (Remix) (with Ariana Grande) - Bonus Track, Streams: 1,736,042,674


doing it now for the 3 artists, putting results in csv

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

artist_urls = {
    "The Weeknd": "https://kworb.net/spotify/artist/1Xyo4u8uXC1ZmMpatF05PJ_songs.html",
    "Lady Gaga": "https://kworb.net/spotify/artist/1HY2Jd0NmPuamShAr6KMms_songs.html",
    "Justin Bieber": "https://kworb.net/spotify/artist/1uNFoZAHBGtllmzznpCI3s_songs.html"
}

def scrape_kworb_data(artist_name, url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for {artist_name}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    titles_selector = ".text a"  # Song titles
    streams_selector = ".sortable td:nth-child(2)"  # Streams

    titles = soup.select(titles_selector)
    streams = soup.select(streams_selector)

    data = []
    for title, stream in zip(titles, streams):
        data.append({
            "artist_name": artist_name,
            "song_name": title.get_text(strip=True),
            "streams": stream.get_text(strip=True)
        })

    return data

all_data = []
for artist_name, url in artist_urls.items():
    print(f"Scraping data for {artist_name}...")
    artist_data = scrape_kworb_data(artist_name, url)
    all_data.extend(artist_data)

df = pd.DataFrame(all_data)

# Remove commas from the 'streams' column and convert it to integer
df['streams'] = df['streams'].str.replace(',', '').astype(int)


df_sorted = df.sort_values(by='streams', ascending=False)


print(df_sorted)


Scraping data for The Weeknd...
Scraping data for Lady Gaga...
Scraping data for Justin Bieber...
       artist_name                                     song_name     streams
0       The Weeknd                               Blinding Lights  4646503363
1       The Weeknd                                       Starboy  3673471314
638  Justin Bieber                     STAY (with Justin Bieber)  3416456759
285      Lady Gaga                          Shallow - Radio Edit  2639367538
639  Justin Bieber                                 Love Yourself  2633767208
..             ...                                           ...         ...
634      Lady Gaga              Poker Face - LLG vs GLG Club Mix      122693
635      Lady Gaga               Just Dance - Space Cowboy Remix      117749
284     The Weeknd   Love Me Harder - Gregor Salto Amsterdam Mix      116844
636      Lady Gaga  Judas - Mirrors Une Autre Monde Mix â Jour      108967
637      Lady Gaga              Marry The Night - Quint

In [18]:
df_sorted['streams'] = df_sorted['streams'].apply(lambda x: f"{x:,}")
df_sorted.to_csv("sorted_streaming_data.csv", index=False)
print("Data saved to 'sorted_streaming_data.csv'")
df_sorted.head()

Data saved to 'sorted_streaming_data.csv'


,artist_name,song_name,streams
0,The Weeknd,Blinding Lights,"4,646,503,363"
1,The Weeknd,Starboy,"3,673,471,314"
638,Justin Bieber,STAY (with Justin Bieber),"3,416,456,759"
285,Lady Gaga,Shallow - Radio Edit,"2,639,367,538"
639,Justin Bieber,Love Yourself,"2,633,767,208"
